In [20]:
import random

def shuffle(arr: list) -> list:
    shuffled_arr = arr.copy()
    random.shuffle(shuffled_arr)
    return shuffled_arr

In [21]:
from dataclasses import asdict, dataclass
from typing import Self


@dataclass
class DESConfiguration:
    ip_table: list[int]
    pc1_table: list[int]
    key_shift_table: list[int]
    pc2_table: list[int]
    e_box_table: list[int]
    s_boxes: list[list[list[int]]]
    p_box_table: list[int]
    ip_inverse_table: list[int]

    def to_dict(self) -> dict:
        return asdict(self)
    
    @classmethod
    def generate(cls) -> Self:
        ip_table = cls._generate_ip_table()
        return cls(
            ip_table=ip_table,
            ip_inverse_table=cls._generate_ip_inverse_table(ip_table),
            pc1_table = cls._generate_pc1_table(),
            key_shift_table= cls._generate_key_shift_table(),
            pc2_table= cls._generate_pc2_table(),
            e_box_table= cls._generate_e_box_table(),
            s_boxes= cls._generate_s_box_tables(),
            p_box_table= cls._generate_p_box_table()
        )
    
    @staticmethod
    def _generate_ip_inverse_table(ip_table: list[int]) -> list[int]:
        ip_inverse_table = ip_table.copy()

        for i, v in enumerate(ip_table):
            ip_inverse_table[v - 1] = i + 1
        return ip_inverse_table
    
    @staticmethod
    def _generate_p_box_table() -> list[int]:
        return shuffle(list(range(1, 33)))

    @staticmethod
    def _generate_ip_table() -> list[int]:
        return shuffle(list(range(1, 65)))

    @staticmethod
    def _generate_pc1_table() -> list[int]:
        return shuffle(list(range(1, 65)))[:56]

    @staticmethod
    def _generate_key_shift_table() -> list[int]:
        return [random.randint(1,2) for _ in range(16)]

    @staticmethod
    def _generate_pc2_table() -> list[int]:
        return shuffle(list(range(1, 57)))[:48]

    @staticmethod
    def _generate_e_box_table() -> list[int]:
        e_box = shuffle(list(range(1, 33)))
        e_box += [random.choice(e_box) for _ in range(16)]
        return e_box

    @staticmethod
    def _generate_s_box_tables() -> list[list[list[int]]]:
        return [[shuffle(list(range(16))) for _ in range(4)] for _ in range(8)]


In [22]:
import hashlib

class DES:
    def __init__(self, config: DESConfiguration) -> None:
        self._config = config

    def encrypt(self, plain_text: str, key: str) -> str:
        plain_text = self._str2bin(plain_text)
        
        if len(plain_text) % 64 != 0:
            plain_text = self._add_padding(plain_text)

        round_keys = self._make_round_keys(key)
        ciphertext = self._encrypt(plain_text, round_keys)
        return ciphertext
    
    def decrypt(self, ciphertext: str, key: str) -> str:
        ciphertext = self._str2bin(ciphertext)

        round_keys = self._make_round_keys(key)[::-1]
        result = self._encrypt(ciphertext, round_keys)
        result = result.replace("|", "")  # remove padding
        return result

    def _add_padding(self, text: str) -> str:    
        bytes_to_add = (64 - len(text) % 64) % 64
        padding = "|"
        text += self._str2bin(padding)*(bytes_to_add // len(self._str2bin(padding)))
        return text

    def _make_round_keys(self, key: str) -> list[str]:
        shorten_key = self._shorten_key(key)
        binary_key = self._str2bin(shorten_key)
        round_keys = self._generate_round_keys(binary_key)
        return round_keys

    def _encrypt(self, text: str, round_keys: list[str]) -> str:
        result = "".join(
            [self._encrypt_block(text[i:i+64], round_keys) for i in range(0, len(text), 64)]
        )
        return self._bin2str(result)

    def _encrypt_block(self, str_bin: str, round_keys: list[str]) -> str:
        ip_result_str = "".join([str_bin[i - 1] for i in self._config.ip_table])

        lpt, rpt = ip_result_str[:32], ip_result_str[32:]
        for round_num in range(16):
            round_key = round_keys[round_num]
            lpt, rpt = self._round(lpt, rpt, round_key)

        final_result = rpt + lpt
        final_cipher = ''.join([final_result[self._config.ip_inverse_table[i] - 1] for i in range(64)])
        return final_cipher
    
    def _round(self, lpt: str, rpt: str, round_key_str: str) -> tuple[str, str]:
        expanded_result_str = "".join([rpt[i - 1] for i in self._config.e_box_table])

        xor_result_str = "".join(
            str(int(expanded_result_str[i]) ^ int(round_key_str[i])) for i in range(48)
        )
        
        six_bit_groups = [xor_result_str[i:i+6] for i in range(0, 48, 6)]
        s_box_substituted = ''
        for i in range(8):
            row_bits = int(six_bit_groups[i][0] + six_bit_groups[i][-1], 2)
            col_bits = int(six_bit_groups[i][1:-1], 2)

            s_box_value = self._config.s_boxes[i][row_bits][col_bits]
            s_box_substituted += format(s_box_value, '04b')
        
        p_box_result_str = ''.join([s_box_substituted[i - 1] for i in self._config.p_box_table])

        new_rpt_str = ''.join([str(int(lpt[i]) ^ int(p_box_result_str[i])) for i in range(32)])
        return rpt, new_rpt_str

    def _generate_round_keys(self, key: str) -> list[str]:
        # 56-bit key in binary
        left = key[:28]
        right = key[28:]
        round_keys = []

        for i in range(16):
            left = left[self._config.key_shift_table[i]:] + left[:self._config.key_shift_table[i]]
            right = right[self._config.key_shift_table[i]:] + right[:self._config.key_shift_table[i]]
            cd_concatenated = left + right

            # make it 48-bit
            round_key = ''.join(cd_concatenated[bit - 1] for bit in self._config.pc2_table)
            round_keys.append(round_key)

        return round_keys

    @staticmethod
    def _bin2str(binary: str) -> str:
        return ''.join([chr(int(binary[i:i+8], 2)) for i in range(0, len(binary), 8)])

    @staticmethod
    def _str2bin(string: str) -> str:
        return "".join(format(ord(char), '08b') for char in string)

    @staticmethod
    def _shorten_key(key: str) -> str:
        # returns 56-bit key
        return hashlib.sha256(key.encode()).hexdigest()[:7]

In [23]:
des_conf = DESConfiguration.generate()
des = DES(des_conf)
encrypted = des.encrypt("Plain Text", "fbit336959")
decrypted = des.decrypt(encrypted, "fbit336959")

print(f"Encrypted: `{encrypted}`")
print(f"Decrypted: `{decrypted}`")

Encrypted: `õëÁ@þ7ë§xØþ¢c`
Decrypted: `Plain Text`
